conda install -c conda-forge google-generativeai

In [2]:
%conda install -c conda-forge google-generativeai


Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [10]:
%pip install grpcio
%pip install python-dotenv



Note: you may need to restart the kernel to use updated packages.


In [11]:

import google.generativeai as genai
import os
import base64

In [18]:
from dotenv import load_dotenv

# Carregar as variáveis do arquivo .env
load_dotenv()

# Acessar a chave da API
API_KEY = os.getenv('GEMINI_AI_API_KEY')

# Configurar o genai
genai.configure(api_key=API_KEY)


In [19]:

def prep_image(image_path):
    # Upload the file and print a confirmation.
    sample_file = genai.upload_file(path=image_path,
                                display_name="Diagram")
    print(f"Uploaded file '{sample_file.display_name}' as: {sample_file.uri}")
    file = genai.get_file(name=sample_file.name)
    print(f"Retrieved file '{file.display_name}' as: {sample_file.uri}")
    return sample_file

In [20]:

def extract_text_from_image(image_path, prompt):
    # Choose a Gemini model.
    model = genai.GenerativeModel(model_name="gemini-1.5-pro")
    # Prompt the model with text and the previously uploaded image.
    response = model.generate_content([image_path, prompt])
    return response.text

In [21]:
sample_file = prep_image('monster.jpg') 
text = extract_text_from_image(sample_file, "Extract the text in the image verbatim")
if text:
    print("Extracted Text:")
    print(text)
else:
    print("Failed to extract text from the image.")

Uploaded file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/pa5vxg6dcgsk
Retrieved file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/pa5vxg6dcgsk
Extracted Text:
**INFORMAÇÃO NUTRICIONAL**

PORÇÕES POR EMBALAGEM: CERCA DE 2
PORÇÃO: 200 ml (1 COPO)

|                         | 100 ml | 200 ml | %VD* |
|-------------------------|---------|---------|-------|
| VALOR ENERGÉTICO (kcal) | 36     | 72     | 4     |
| CARBOIDRATOS (g)       | 9,2    | 18     | 6     |
| AÇÚCARES TOTAIS (g)    | 8,3    | 17     |       |
| AÇÚCARES ADICIONADOS (g)| 8,3    | 17     | 34    |
| SÓDIO (mg)              | 79     | 158    | 8     |
| VITAMINA B3 (mg)        | 7,5    | 15     | 100   |
| VITAMINA B6 (mg)        | 0,65   | 1,3    | 100   |
| VITAMINA B2 (mg)        | 0,6    | 1,2    | 100   |
| VITAMINA B12 (µg)       | 1,2    | 2,4    | 100   |
| TAURINA (mg)           | 400    | 800    |       |
| CAFEÍNA (mg)           | 33     | 66     |       |
| INO

In [31]:
!pip install crewai langchain-google-genai crewai_tools

  Using cached crewai-0.86.0-py3-none-any.whl.metadata (19 kB)
  Using cached crewai_tools-0.17.0-py3-none-any.whl.metadata (4.9 kB)
  Using cached auth0_python-4.7.2-py3-none-any.whl.metadata (8.9 kB)
  Using cached chromadb-0.5.23-py3-none-any.whl.metadata (6.8 kB)
  Using cached jsonref-1.1.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached opentelemetry_api-1.29.0-py3-none-any.whl.metadata (1.4 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.29.0-py3-none-any.whl.metadata (2.2 kB)
  Using cached opentelemetry_sdk-1.29.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pdfplumber-0.11.4-py3-none-any.whl.metadata (41 kB)
  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
  Using cached pyvis-0.3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached tomli_w-1.1.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached tomli-2.2.1-cp312-cp312-win_amd64.whl.metadata (12 kB)
  Using cached docker-7.1.0-py3-none-any.whl.metadata (3.8 kB)
  Using cached docx2txt-0.8

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.


In [36]:
import os
from dotenv import load_dotenv
load_dotenv()

from crewai import Agent, Task, Crew
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import LLM
import litellm

# Configuração da API
litellm.api_key = API_KEY
GEMINI_API_KEY = API_KEY  # Redundância para manter o comportamento original
llm = LLM(
    model="gemini/gemini-pro",
    temperature=0.7,
    google_api_key=API_KEY,
    tools='google_search_retrieval',
)

# Avisos
import warnings
warnings.filterwarnings('ignore')

# Função de análise de texto nutricional
def analyze_nutrition_label(label_text, output_file="nutrition_analysis.md"):
    # Definir o tópico como o texto do rótulo extraído
    inputs_array = {'topic': label_text}

    # Atualizar o agente para o papel de nutricionista
    nutritionist_agent = Agent(
        role="Nutricionista",
        goal="Analisar um rótulo nutricional e fornecer uma análise detalhada dos pontos positivos e negativos em tópicos.",
        verbose=True,
        memory=True,
        backstory=(
            "Você é um nutricionista experiente. Sua tarefa é analisar o texto fornecido de um rótulo nutricional, "
            "identificar seus benefícios e potenciais riscos à saúde, e apresentar um resumo claro em tópicos."
        ),
        llm=llm,
    )

    # Tarefa do nutricionista
    nutrition_task = Task(
        description=(
            "Leia o texto do rótulo nutricional fornecido e analise seus pontos positivos e negativos."
            "Forneça uma lista clara e detalhada dos prós e contras relacionados aos valores nutricionais, ingredientes e outros aspectos importantes."
        ),
        expected_output="Uma lista em tópicos com os pontos positivos e negativos do rótulo nutricional.",
        agent=nutritionist_agent,
        output_file=output_file,
    )

    # Criar a equipe
    crew = Crew(
        agents=[nutritionist_agent],
        tasks=[nutrition_task],
        verbose=True
    )

    # Executar a análise
    crew_output = crew.kickoff(inputs=inputs_array)

    return crew_output

# Função principal
if __name__ == "__main__":
    # Substituir por sua função de extração de texto
    sample_file = prep_image('monster.jpg')
    label_text = extract_text_from_image(sample_file, "Extract the text in the image verbatim")

    if label_text:
        print("Texto extraído do rótulo:")
        print(label_text)

        # Análise do Nutricionista e salvamento em Markdown
        output_file = "nutrition_analysis.md"
        analyze_nutrition_label(label_text, output_file=output_file)

        print(f"\nAnálise salva no arquivo: {output_file}")
    else:
        print("Falha ao extrair texto da imagem.")


Uploaded file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/zgr5gw0g8dmq
Retrieved file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/zgr5gw0g8dmq


Overriding of current TracerProvider is not allowed


Texto extraído do rótulo:
**INFORMAÇÃO NUTRICIONAL**

PORÇÕES POR EMBALAGEM: CERCA DE 2
PORÇÃO: 200 ml (1 COPO)

| | 100 ml | 200 ml | %VD* |
|---|---|---|---|
| VALOR ENERGÉTICO (kcal) | 36 | 72 | 4 |
| CARBOIDRATOS (g) | 9,2 | 18 | 6 |
| AÇÚCARES TOTAIS (g) | 8,3 | 17 |  |
| AÇÚCARES ADICIONADOS (g) | 8,3 | 17 | 34 |
| SÓDIO (mg) | 79 | 158 | 8 |
| VITAMINA B3 (mg) | 7,5 | 15 | 100 |
| VITAMINA B6 (mg) | 0,65 | 1,3 | 100 |
| VITAMINA B2 (mg) | 0,6 | 1,2 | 100 |
| VITAMINA B12 (µg) | 1,2 | 2,4 | 100 |
| TAURINA (mg) | 400 | 800 |  |
| CAFEÍNA (mg) | 33 | 66 |  |
| INOSITOL (mg) | 2 | 4 |  |


PERCENTUAL DE VALORES DIÁRIOS FORNECIDOS PELA PORÇÃO

NÃO CONTÉM QUANTIDADES SIGNIFICATIVAS DE: PROTEÍNAS, GORDURAS TOTAIS, GORDURAS SATURADAS, GORDURAS TRANS E FIBRAS ALIMENTARES.

INGREDIENTES: ÁGUA GASEIFICADA, AÇÚCAR,...
# Agent: Nutricionista
## Task: Leia o texto do rótulo nutricional fornecido e analise seus pontos positivos e negativos.Forneça uma lista clara e detalhada dos prós e contra

In [30]:
sample_file = prep_image('jetpack2.jpg') 
text = extract_text_from_image(sample_file, "Analyze the given diagram and carefully extract the information. Include the cost of the item")
if text:
    print("Interpreted Image:")
    print(text)
else:
    print("Failed to extract text from the image.")

Uploaded file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/dzj5sivj4fit
Retrieved file 'Diagram' as: https://generativelanguage.googleapis.com/v1beta/files/dzj5sivj4fit
Interpreted Image:
The diagram depicts a conceptual "Jetpack Backpack" with the following features:

* **Fits 18" Laptop:**  Sized to accommodate a standard large laptop.
* **Padded Strap Support:** Designed for comfortable carrying.
* **Lightweight; Looks Like a Normal Backpack:**  Discreet and practical for everyday use.
* **USB-C Charging:** Modern charging capability.
* **15-Min Battery Life:**  Short burst of jetpack functionality.
* **Retractible Boosters:** The jetpack mechanism can be hidden when not in use.
* **Steam-Powered, Green/Clean:**  Eco-friendly propulsion method.
* **Price: $399.99**

